# PDI - Rebalancing data for July 2022

## Setting

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
from defillama import DefiLlama
llama = DefiLlama()

In [4]:
import plotly.express as px
import plotly.graph_objects as go

In [5]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics** 
    1. The project must have a listing on Defi Llama
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.

1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have at least $5mm of total liquidity at Uniswap v2, Uniswap v3, Sushiswap, Balancer, Bancor. 
    1. The token must have shown consistent DeFi liquidity on Ethereum.

1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [6]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### DefiLlama

In [7]:
protocols = llama.get_all_protocols()

In [8]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = normalize_symb(pr['symbol']) 
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': pr['address'].lower()
        }

#### CoinGecko

##### Tokens addresses

In [9]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [10]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = normalize_symb(coin['symbol'])
        address = coin['platforms']['ethereum'].lower()
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token

In [11]:
tokens['USDC'], tokens['WETH']

({'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'})

In [12]:
tokens['BNT']

{'symbol': 'BNT', 'address': '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c'}

In [13]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        print(token)
        exclude_symbols.append(token['symbol'])

{'symbol': 'MULTI', 'address': '0x9fb9a33956351cf4fa040f65a13b835a3c8764e3'}
{'symbol': 'IB', 'address': 'fantom:0x00a35fd824c717879bf370e70ac6868b95870dfb'}
{'symbol': 'COW', 'address': 'heco:0x80861a817106665bca173db6ac2ab628a738c737'}
{'symbol': 'PINKSALE', 'address': 'bsc:0x602ba546a7b06e0fc7f58fd27eb6996ecc824689'}
{'symbol': 'ABR', 'address': 'celo:0x6e512bfc33be36f2666754e996ff103ad1680cc9'}
{'symbol': 'FLAG', 'address': '0x57dd84042ec9507963016596a34edad42f7e4cce '}
{'symbol': 'HEC', 'address': 'fantom:0x5c4fdfc5233f935f20d2adba572f770c2e377ab0'}
{'symbol': 'RSR', 'address': '0x8762db106b2c2a0bccb3a80d1ed41273552616e8'}
{'symbol': 'FHM', 'address': '0xfa1fbb8ef55a4855e5688c0ee13ac3f202486286'}
{'symbol': 'SGT', 'address': '0x84810bcf08744d5862b8181f12d17bfd57d3b078'}
{'symbol': 'GEAR', 'address': '0xba3335588d9403515223f109edc4eb7269a9ab5d'}
{'symbol': 'SOLV', 'address': '0x256f2d67e52fe834726d2ddcd8413654f5eb8b53'}
{'symbol': 'SWPR', 'address': 'arbitrum:0xde903e2712288a1da829

In [14]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

In [15]:
# KEEP has this address in defillama, but etherscan says this address belongs to tBTC now
# for token in top_defi_tokens.values():
#     if token['address'] == "0x8daebade922df735c38c80c7ebd708af50815faa":
#         print(token)
# del top_defi_tokens['KEEP']

##### Market data

In [16]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [17]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())

In [18]:
duplicated = tokens_market_data.index[tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.drop(index=duplicated)

In [19]:
tokens_market_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
BTC,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,19776.360000,377583245130,1,4.153036e+11,2.162735e+10,19994.110000,...,2.100000e+07,2.100000e+07,69045.000000,-71.35084,2021-11-10T14:24:11.849Z,67.810000,29071.26869,2013-07-06T00:00:00.000Z,None,2022-07-13T11:29:51.228Z
ETH,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1073.480000,128304213284,2,NaN,1.198907e+10,1086.660000,...,1.196641e+08,NaN,4878.260000,-78.00754,2021-11-10T14:24:19.604Z,0.432979,247683.37408,2015-10-20T00:00:00.000Z,"{'times': 71.53735571905364, 'currency': 'btc'...",2022-07-13T11:31:16.072Z
USDT,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999773,65935969752,3,NaN,4.579968e+10,1.008000,...,6.595093e+10,NaN,1.320000,-24.47700,2018-07-24T00:00:00.000Z,0.572521,74.53357,2015-03-02T00:00:00.000Z,None,2022-07-13T11:26:06.481Z
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.001000,55309591798,4,NaN,3.843216e+09,1.007000,...,5.527281e+10,NaN,1.170000,-14.71908,2019-05-08T00:40:28.300Z,0.891848,12.13734,2021-05-19T13:14:05.611Z,None,2022-07-13T11:31:19.874Z
BNB,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,225.970000,36893491655,5,3.730920e+10,9.221243e+08,228.150000,...,1.632770e+08,1.651168e+08,686.310000,-67.05901,2021-05-10T07:24:17.097Z,0.039818,567678.12910,2017-10-19T00:00:00.000Z,None,2022-07-13T11:30:13.698Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SYNC,sync-network,sync,Sync Network,https://assets.coingecko.com/coins/images/1323...,0.008306,991144,1893,NaN,1.750070e+03,0.008332,...,1.542655e+08,NaN,0.209893,-96.04286,2021-03-08T12:42:14.073Z,0.002878,188.60792,2020-12-12T05:24:34.909Z,None,2022-07-13T11:20:08.408Z
BED,bankless-bed-index,bed,Bankless BED Index,https://assets.coingecko.com/coins/images/1717...,46.500000,990098,1895,NaN,3.763470e+03,46.760000,...,2.129724e+04,NaN,198.730000,-76.62414,2021-11-10T14:23:50.390Z,36.640000,26.78795,2022-06-18T20:57:24.718Z,None,2022-07-13T11:31:35.968Z
LEOS,leonicorn-swap,leos,Leonicorn Swap,https://assets.coingecko.com/coins/images/1689...,0.157378,987757,1896,NaN,5.792250e+05,0.172018,...,2.096993e+08,NaN,2.030000,-92.71679,2021-10-19T20:35:24.195Z,0.057493,157.39006,2021-07-08T14:02:11.725Z,None,2022-07-13T11:31:28.270Z


### Liquidity check

In [20]:
api_urls = {
    'univ2': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    'univ3': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3",
    'sushi': "https://api.thegraph.com/subgraphs/name/sushiswap/exchange",
    'bancorv2': "https://api-v2.bancor.network/tokens",
    'bancorv3': "https://api-v3.bancor.network/pools",
    'balancer': "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer",
    'balancerv2': "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-v2" 
} 

In [21]:
def univ2_get_liquidities(tokens_ids):
    query = """
        query tokens ($tokens_ids: [ID!]!) {
            tokens (first:1000, where: {id_in: $tokens_ids}){
                id
                symbol
                totalLiquidity
            }
        }
    """
    url = api_urls['univ2']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    tokens = response.json()['data']['tokens']
    
    liqs = {}
    for token in tokens:
        symb = normalize_symb(token['symbol'])
        liqs[symb] = float(token['totalLiquidity'])
        
    return liqs
    

In [22]:
def univ3_get_liquidities(tokens_ids):
    query = """
        query tokens ($tokens_ids: [ID!]!) {
            tokens (first:1000, where: {id_in: $tokens_ids}){
                id
                symbol
                totalValueLockedUSD
            }
        }
    """
    url = api_urls['univ3']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    tokens = response.json()['data']['tokens']
    
    liqs = {}
    for token in tokens:
        symb = normalize_symb(token['symbol'])
        liqs[symb] = float(token['totalValueLockedUSD'])
        
    return liqs
    

In [23]:
def sushi_get_liquidities(tokens_ids):
#     query = """
#         query tokens ($tokens_ids: [id!]!) {
#             tokens (first:1000, where: {id_in: $tokens_ids}){
#                 id
#                 symbol
#                 liquidity
#             }
#         }
#     """
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, where: {token0_in: $tokens_ids, token1_in: $tokens_ids}){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserve0
                reserve1
            }
        }
    """
    url = api_urls['sushi']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        symb0 = normalize_symb(pair['token0']['symbol'])
        if symb0 in tokens_market_data.index and pair['token0']['id'].lower() in tokens_ids:
            liqs[symb0] = liqs.get(symb0, 0) + float(pair['reserve0']) * tokens_market_data.loc[symb0]['current_price']
        
        symb1 = normalize_symb(pair['token1']['symbol'])
        if symb1 in tokens_market_data.index and pair['token1']['id'].lower() in tokens_ids:
            liqs[symb1] = liqs.get(symb1, 0) + float(pair['reserve1']) * tokens_market_data.loc[symb1]['current_price']
        
    return liqs
    

In [24]:
def bancorv2_get_liquidities(tokens_ids):
    url = api_urls['bancorv2'] 
    response = requests.get(url)
    tokens = response.json()['data']
    
    liqs = {}
    for token in tokens:
        if token['dlt_id'].lower() in tokens_ids:
            symb = normalize_symb(token['symbol'])
            liqs[symb] = float(token['liquidity']['usd'])
            
    return liqs
            

In [25]:
def bancorv3_get_liquidities(tokens_ids):
    url = api_urls['bancorv3'] 
    response = requests.get(url)
    pools = response.json()['data']
    
    liqs = {}
    for pool in pools:
        if pool['poolDltId'].lower() in tokens_ids:
            symb = normalize_symb(pool['name'])
            liqs[symb] = liqs.get(symb, 0) + float(pool['liquidity']['usd'])
            
    return liqs
            

In [26]:
def balancer_get_liquidities(tokens_ids):
    query = """
        query pools {
            pools (
                first: 1000,
                where: {
                  liquidity_gt: 10000,
                  
                }
            ){
                tokens {
                  address
                  symbol
                  balance
                }
          }
        }
    """
    url = api_urls['balancer']
    
    response = requests.post(url, json={'query': query})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb not in tokens_market_data.index or token['address'].lower() not in tokens_ids:
                continue
            
            liqs[symb] = liqs.get(symb, 0) + float(token['balance']) * tokens_market_data.loc[symb]['current_price']
        
    return liqs
    

In [27]:
def balancerv2_get_liquidities(tokens_ids):
    query = """
        query pools {
            pools (
                first: 1000,
                where: {
                  totalLiquidity_gt: 10000,
                }
            ){
                tokens {
                  address
                  symbol
                  balance
                }
          }
        }
    """
    url = api_urls['balancerv2']
    
    response = requests.post(url, json={'query': query})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb not in tokens_market_data.index or token['address'].lower() not in tokens_ids:
                continue
            
            liqs[symb] = liqs.get(symb, 0) + float(token['balance']) * tokens_market_data.loc[symb]['current_price']
        
    return liqs
    

In [28]:
tokens_ids = [token['address'] for symb, token in top_defi_tokens.items()]

In [29]:
tokens_ids.append(tokens['WETH']['address'])
tokens_ids.append(tokens['USDC']['address'])

In [30]:
query_functions = {
    'univ2': univ2_get_liquidities,
    'univ3': univ3_get_liquidities, 
    'sushi': sushi_get_liquidities, 
    'bancorv2': bancorv2_get_liquidities,
    'bancorv3': bancorv3_get_liquidities,
    'balancer': balancer_get_liquidities, 
    'balancerv2': balancerv2_get_liquidities, 
}

In [31]:
liqs_dict = {}
for dex, func in query_functions.items():
    liqs_dict[dex] = func(tokens_ids)

In [32]:
liquidities = pd.DataFrame.from_dict(liqs_dict)

In [33]:
liquidities['sum'] = liquidities.sum(axis=1)
liquidities.sort_values('sum', ascending=False)

,univ2,univ3,sushi,bancorv2,bancorv3,balancer,balancerv2,sum
BAO,1.459662e+09,0.000000e+00,2.947821e+05,NaN,NaN,NaN,NaN,1.459957e+09
USDC,1.891922e+08,1.096398e+09,2.304520e+07,1.025826e+06,1.173977e+06,8.915539e+06,1.931388e+07,1.339065e+09
WETH,3.554478e+05,7.838456e+08,NaN,NaN,NaN,NaN,NaN,7.842010e+08
LFT,6.304008e+08,0.000000e+00,NaN,NaN,NaN,NaN,NaN,6.304008e+08
RISE,6.074114e+08,NaN,NaN,NaN,NaN,NaN,NaN,6.074114e+08
...,...,...,...,...,...,...,...,...
VIRTUE,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00
BIOS,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00
FOREX,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00
ITG,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000000e+00


In [34]:
liquidities.loc['YFI']

univ2         2.623823e+01
univ3         2.784137e+05
sushi         2.415921e+06
bancorv2      5.024991e+05
bancorv3      0.000000e+00
balancer      2.715491e+03
balancerv2    2.973026e+06
sum           6.172601e+06
Name: YFI, dtype: float64

In [35]:
liq_check = (liquidities['sum'] > 5e6).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [36]:
lq_passed = tokens_market_data[
    tokens_market_data.index.isin(liq_check_passed.index)
]

In [37]:
lq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.001000,55309591798,4,NaN,3.843216e+09,1.007000,...,5.527281e+10,NaN,1.170000,-14.71908,2019-05-08T00:40:28.300Z,8.918480e-01,1.213734e+01,2021-05-19T13:14:05.611Z,None,2022-07-13T11:31:19.874Z
WBTC,wrapped-bitcoin,wbtc,Wrapped Bitcoin,https://assets.coingecko.com/coins/images/7598...,19779.270000,4909898698,17,4.909899e+09,2.778767e+08,19982.110000,...,2.483826e+05,2.483826e+05,70643.000000,-72.00053,2021-11-10T14:40:19.650Z,3.139170e+03,5.300951e+02,2019-04-02T00:00:00.000Z,None,2022-07-13T11:30:55.488Z
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.620000,2567047059,26,5.623452e+09,1.579604e+08,5.860000,...,1.000000e+09,1.000000e+09,44.920000,-87.49901,2021-05-03T05:25:04.822Z,1.030000e+00,4.450749e+02,2020-09-17T01:20:38.214Z,None,2022-07-13T11:31:16.562Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,72.190000,1005243342,51,1.153728e+09,2.622095e+08,73.260000,...,1.600000e+07,1.600000e+07,661.690000,-89.08915,2021-05-18T21:19:59.514Z,2.602000e+01,1.774301e+02,2020-11-05T09:20:11.928Z,None,2022-07-13T11:30:52.014Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,852.990000,767757679,59,8.565739e+08,6.430530e+07,873.200000,...,9.776310e+05,1.005577e+06,6292.310000,-86.44423,2021-05-03T21:54:29.333Z,1.683600e+02,4.066429e+02,2020-03-16T20:52:36.527Z,None,2022-07-13T11:30:44.248Z
AMP,amp-token,amp,Amp,https://assets.coingecko.com/coins/images/1240...,0.008885,426640197,93,8.816909e+08,7.036867e+06,0.009088,...,9.944412e+10,9.922516e+10,0.120813,-92.64988,2021-06-16T16:03:41.147Z,2.505010e-03,2.544842e+02,2020-11-04T13:18:12.191Z,None,2022-07-13T11:30:49.637Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,0.815587,397010175,98,8.165004e+08,2.884178e+07,0.821066,...,1.000000e+09,1.000000e+09,7.300000,-89.18888,2021-08-20T08:35:20.158Z,4.061500e-01,9.439183e+01,2022-06-18T20:55:12.035Z,None,2022-07-13T11:31:02.514Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,4.810000,344564772,107,4.803811e+08,2.889675e+06,4.920000,...,9.982298e+07,9.982298e+07,42.800000,-88.76784,2022-01-12T15:22:27.465Z,1.500000e+00,2.198155e+02,2021-06-25T16:50:51.447Z,None,2022-07-13T11:31:43.024Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,47.580000,325691285,111,4.750397e+08,9.811240e+07,48.740000,...,1.000000e+07,1.000000e+07,910.540000,-94.76587,2021-05-12T02:29:08.794Z,2.641000e+01,8.045679e+01,2022-06-18T20:54:18.261Z,None,2022-07-13T11:31:16.317Z


### Non quantative checks

In [38]:
exclude_list = [
    {
        'symbol': 'WBTC',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'FXS',
        'reason': 'Criteria 1.G'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BOND',
        'reason': 'Criteria 1.I'
    },
    {
        'symbol': 'USDC',
        'reason': 'stable'
    },
    {
        'symbol': 'FEI',
        'reason': 'stable'
    }
]
exclude_symbols = [token['symbol'] for token in exclude_list]

In [39]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_symbols)
]

In [40]:
nq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.620000,2567047059,26,5.623452e+09,1.579604e+08,5.860000,...,1.000000e+09,1.000000e+09,44.920000,-87.49901,2021-05-03T05:25:04.822Z,1.030000e+00,4.450749e+02,2020-09-17T01:20:38.214Z,None,2022-07-13T11:31:16.562Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,72.190000,1005243342,51,1.153728e+09,2.622095e+08,73.260000,...,1.600000e+07,1.600000e+07,661.690000,-89.08915,2021-05-18T21:19:59.514Z,2.602000e+01,1.774301e+02,2020-11-05T09:20:11.928Z,None,2022-07-13T11:30:52.014Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,852.990000,767757679,59,8.565739e+08,6.430530e+07,873.200000,...,9.776310e+05,1.005577e+06,6292.310000,-86.44423,2021-05-03T21:54:29.333Z,1.683600e+02,4.066429e+02,2020-03-16T20:52:36.527Z,None,2022-07-13T11:30:44.248Z
AMP,amp-token,amp,Amp,https://assets.coingecko.com/coins/images/1240...,0.008885,426640197,93,8.816909e+08,7.036867e+06,0.009088,...,9.944412e+10,9.922516e+10,0.120813,-92.64988,2021-06-16T16:03:41.147Z,2.505010e-03,2.544842e+02,2020-11-04T13:18:12.191Z,None,2022-07-13T11:30:49.637Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,0.815587,397010175,98,8.165004e+08,2.884178e+07,0.821066,...,1.000000e+09,1.000000e+09,7.300000,-89.18888,2021-08-20T08:35:20.158Z,4.061500e-01,9.439183e+01,2022-06-18T20:55:12.035Z,None,2022-07-13T11:31:02.514Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,47.580000,325691285,111,4.750397e+08,9.811240e+07,48.740000,...,1.000000e+07,1.000000e+07,910.540000,-94.76587,2021-05-12T02:29:08.794Z,2.641000e+01,8.045679e+01,2022-06-18T20:54:18.261Z,None,2022-07-13T11:31:16.317Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.170000,226090124,144,2.931830e+08,5.420744e+07,1.190000,...,2.431301e+08,2.500000e+08,23.380000,-94.96893,2021-03-13T23:44:36.774Z,4.753810e-01,1.473986e+02,2020-11-04T14:53:53.560Z,None,2022-07-13T11:31:34.792Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,5644.440000,178134839,172,2.070340e+08,3.558904e+07,5754.590000,...,3.666600e+04,3.666600e+04,90787.000000,-93.78563,2021-05-12T00:29:37.713Z,3.165000e+01,1.772843e+04,2020-07-18T12:26:27.150Z,None,2022-07-13T11:31:43.201Z
BAL,balancer,bal,Balancer,https://assets.coingecko.com/coins/images/1168...,4.270000,148763752,187,4.108712e+08,2.047017e+07,4.400000,...,5.078425e+07,9.615070e+07,74.450000,-94.26085,2021-05-04T13:35:02.939Z,3.660000e+00,1.687948e+01,2022-06-18T21:00:00.231Z,None,2022-07-13T11:31:34.265Z


### Retrieve historical data & livetime check

In [41]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Note: RISE, marketcap data available only for 279 < 365 days
Excluding POP, prices data available only for 214 < 365 days
Excluding TOKE, prices data available only for 341 < 365 days
Excluding ALD, prices data available only for 307 < 365 days


In [42]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [43]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.620000,2567047059,26,5.623452e+09,1.579604e+08,5.860000,...,1.000000e+09,1.000000e+09,44.920000,-87.49901,2021-05-03T05:25:04.822Z,1.030000e+00,4.450749e+02,2020-09-17T01:20:38.214Z,None,2022-07-13T11:31:16.562Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,72.190000,1005243342,51,1.153728e+09,2.622095e+08,73.260000,...,1.600000e+07,1.600000e+07,661.690000,-89.08915,2021-05-18T21:19:59.514Z,2.602000e+01,1.774301e+02,2020-11-05T09:20:11.928Z,None,2022-07-13T11:30:52.014Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,852.990000,767757679,59,8.565739e+08,6.430530e+07,873.200000,...,9.776310e+05,1.005577e+06,6292.310000,-86.44423,2021-05-03T21:54:29.333Z,1.683600e+02,4.066429e+02,2020-03-16T20:52:36.527Z,None,2022-07-13T11:30:44.248Z
AMP,amp-token,amp,Amp,https://assets.coingecko.com/coins/images/1240...,0.008885,426640197,93,8.816909e+08,7.036867e+06,0.009088,...,9.944412e+10,9.922516e+10,0.120813,-92.64988,2021-06-16T16:03:41.147Z,2.505010e-03,2.544842e+02,2020-11-04T13:18:12.191Z,None,2022-07-13T11:30:49.637Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,0.815587,397010175,98,8.165004e+08,2.884178e+07,0.821066,...,1.000000e+09,1.000000e+09,7.300000,-89.18888,2021-08-20T08:35:20.158Z,4.061500e-01,9.439183e+01,2022-06-18T20:55:12.035Z,None,2022-07-13T11:31:02.514Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,47.580000,325691285,111,4.750397e+08,9.811240e+07,48.740000,...,1.000000e+07,1.000000e+07,910.540000,-94.76587,2021-05-12T02:29:08.794Z,2.641000e+01,8.045679e+01,2022-06-18T20:54:18.261Z,None,2022-07-13T11:31:16.317Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.170000,226090124,144,2.931830e+08,5.420744e+07,1.190000,...,2.431301e+08,2.500000e+08,23.380000,-94.96893,2021-03-13T23:44:36.774Z,4.753810e-01,1.473986e+02,2020-11-04T14:53:53.560Z,None,2022-07-13T11:31:34.792Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,5644.440000,178134839,172,2.070340e+08,3.558904e+07,5754.590000,...,3.666600e+04,3.666600e+04,90787.000000,-93.78563,2021-05-12T00:29:37.713Z,3.165000e+01,1.772843e+04,2020-07-18T12:26:27.150Z,None,2022-07-13T11:31:43.201Z
BAL,balancer,bal,Balancer,https://assets.coingecko.com/coins/images/1168...,4.270000,148763752,187,4.108712e+08,2.047017e+07,4.400000,...,5.078425e+07,9.615070e+07,74.450000,-94.26085,2021-05-04T13:35:02.939Z,3.660000e+00,1.687948e+01,2022-06-18T21:00:00.231Z,None,2022-07-13T11:31:34.265Z


### Marketcap ranking & top10 filtering

In [44]:
chosen_tokens = lt_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [45]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.620000,2567047059,26,5.623452e+09,157960398.0,5.860000,...,1.000000e+09,1.000000e+09,44.920000,-87.49901,2021-05-03T05:25:04.822Z,1.030000,445.07490,2020-09-17T01:20:38.214Z,None,2022-07-13T11:31:16.562Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,72.190000,1005243342,51,1.153728e+09,262209527.0,73.260000,...,1.600000e+07,1.600000e+07,661.690000,-89.08915,2021-05-18T21:19:59.514Z,26.020000,177.43006,2020-11-05T09:20:11.928Z,None,2022-07-13T11:30:52.014Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,852.990000,767757679,59,8.565739e+08,64305305.0,873.200000,...,9.776310e+05,1.005577e+06,6292.310000,-86.44423,2021-05-03T21:54:29.333Z,168.360000,406.64288,2020-03-16T20:52:36.527Z,None,2022-07-13T11:30:44.248Z
AMP,amp-token,amp,Amp,https://assets.coingecko.com/coins/images/1240...,0.008885,426640197,93,8.816909e+08,7036867.0,0.009088,...,9.944412e+10,9.922516e+10,0.120813,-92.64988,2021-06-16T16:03:41.147Z,0.002505,254.48416,2020-11-04T13:18:12.191Z,None,2022-07-13T11:30:49.637Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,0.815587,397010175,98,8.165004e+08,28841778.0,0.821066,...,1.000000e+09,1.000000e+09,7.300000,-89.18888,2021-08-20T08:35:20.158Z,0.406150,94.39183,2022-06-18T20:55:12.035Z,None,2022-07-13T11:31:02.514Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,47.580000,325691285,111,4.750397e+08,98112396.0,48.740000,...,1.000000e+07,1.000000e+07,910.540000,-94.76587,2021-05-12T02:29:08.794Z,26.410000,80.45679,2022-06-18T20:54:18.261Z,None,2022-07-13T11:31:16.317Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.170000,226090124,144,2.931830e+08,54207440.0,1.190000,...,2.431301e+08,2.500000e+08,23.380000,-94.96893,2021-03-13T23:44:36.774Z,0.475381,147.39863,2020-11-04T14:53:53.560Z,None,2022-07-13T11:31:34.792Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,5644.440000,178134839,172,2.070340e+08,35589045.0,5754.590000,...,3.666600e+04,3.666600e+04,90787.000000,-93.78563,2021-05-12T00:29:37.713Z,31.650000,17728.42548,2020-07-18T12:26:27.150Z,None,2022-07-13T11:31:43.201Z
BAL,balancer,bal,Balancer,https://assets.coingecko.com/coins/images/1168...,4.270000,148763752,187,4.108712e+08,20470173.0,4.400000,...,5.078425e+07,9.615070e+07,74.450000,-94.26085,2021-05-04T13:35:02.939Z,3.660000,16.87948,2022-06-18T21:00:00.231Z,None,2022-07-13T11:31:34.265Z


In [46]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [47]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [48]:
prices_data

,UNI,AAVE,MKR,AMP,LDO,COMP,SUSHI,YFI,BAL,BNT
date,,,,,,,,,,
2021-02-10,19.513947,490.595917,2566.847232,0.012622,2.521327,487.702972,14.324851,35097.258976,38.884601,2.811974
2021-02-11,20.606185,524.806338,2514.308324,0.018005,2.163042,475.521807,14.890322,35967.768454,42.514573,2.754296
2021-02-12,21.473246,517.005675,2528.296118,0.030322,2.270908,501.255182,16.879822,43789.391440,47.315055,3.158194
2021-02-13,22.702256,529.260260,2739.946179,0.034258,2.340126,535.007646,16.574149,47117.151242,52.166919,3.489128
2021-02-14,21.772030,493.773782,2601.196210,0.036486,2.533633,510.631578,15.955116,47063.563316,48.474940,3.631959
...,...,...,...,...,...,...,...,...,...,...
2022-07-09,5.853430,73.217665,978.230295,0.009236,0.744939,47.603244,1.170716,6409.936579,5.076565,0.506647
2022-07-10,6.473936,81.915545,997.095484,0.009137,0.681700,55.441627,1.243223,6503.149661,5.038602,0.505566
2022-07-11,6.255603,77.842605,939.754249,0.009168,0.640254,53.282337,1.313602,6035.018233,4.631000,0.483669


In [49]:
marketcaps

,UNI,AAVE,MKR,AMP,LDO,COMP,SUSHI,YFI,BAL,BNT
date,,,,,,,,,,
2021-02-10,5.848772e+09,6.082196e+09,2.321050e+09,4.532621e+08,2.231614e+07,2.099922e+09,1.833098e+09,1.106517e+09,4.201539e+08,3.172505e+08
2021-02-11,6.189234e+09,6.507804e+09,2.267010e+09,6.612278e+08,1.945476e+07,2.059282e+09,1.889832e+09,1.139469e+09,4.545828e+08,3.124235e+08
2021-02-12,6.429248e+09,6.401771e+09,2.271078e+09,1.074686e+09,3.127728e+07,2.160530e+09,2.154083e+09,1.379553e+09,5.089683e+08,3.557003e+08
2021-02-13,6.815509e+09,6.544992e+09,2.452029e+09,1.231475e+09,3.240102e+07,2.297470e+09,2.110044e+09,1.488491e+09,5.631770e+08,3.949681e+08
2021-02-14,6.540139e+09,6.147532e+09,2.354080e+09,1.335279e+09,3.411206e+07,2.211330e+09,2.037300e+09,1.493759e+09,5.251610e+08,4.091548e+08
...,...,...,...,...,...,...,...,...,...,...
2022-07-09,2.685487e+09,1.023467e+09,8.820992e+08,4.438580e+08,3.543692e+08,3.259081e+08,2.266397e+08,2.026104e+08,1.762983e+08,1.148192e+08
2022-07-10,2.960166e+09,1.140935e+09,8.997875e+08,4.369258e+08,3.268671e+08,3.804981e+08,2.396836e+08,2.051187e+08,1.753070e+08,1.151592e+08
2022-07-11,2.855618e+09,1.082198e+09,8.466639e+08,4.396093e+08,3.083165e+08,3.647605e+08,2.520503e+08,1.902320e+08,1.609163e+08,1.092933e+08


## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [50]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [51]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

/home/evgeniy/.local/lib/python3.10/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [52]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [53]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [54]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
adjusted_weights

,UNI,AAVE,MKR,AMP,COMP,LDO,SUSHI,YFI,BAL,BNT
date,,,,,,,,,,
2021-02-10,0.285554,0.296950,0.113320,0.022130,0.102524,0.000000,0.089497,0.054023,0.020513,0.015489
2021-02-11,0.289351,0.300000,0.105984,0.030913,0.096273,0.000000,0.088351,0.053271,0.021252,0.014606
2021-02-12,0.282783,0.281575,0.099891,0.047269,0.095028,0.000000,0.094745,0.060678,0.022386,0.015645
2021-02-13,0.285190,0.273870,0.102603,0.051530,0.096136,0.000000,0.088293,0.062285,0.023566,0.016527
2021-02-14,0.283691,0.266661,0.102113,0.057920,0.095921,0.000000,0.088372,0.064795,0.022780,0.017748
...,...,...,...,...,...,...,...,...,...,...
2022-07-09,0.300000,0.191044,0.164655,0.082852,0.060835,0.066148,0.042305,0.037820,0.032908,0.021433
2022-07-10,0.300000,0.203724,0.160665,0.078017,0.067941,0.058365,0.042798,0.036626,0.031303,0.020563
2022-07-11,0.300000,0.201793,0.157874,0.081972,0.068015,0.057490,0.046999,0.035472,0.030005,0.020379


In [ ]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

/home/evgeniy/.local/lib/python3.10/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [ ]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [ ]:
last_weights = adjusted_weights.iloc[-1].dropna()
last_weights

In [ ]:
converted_last_weights = convert_weights(last_weights)
converted_last_weights

## Summary table

In [ ]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)
pdi